# How to use the python API to run a simple random allocation simulation

This examples shows how to use the python API to:

1. Create a fleet of electric vehicles
1. Create a charging-post infrastructure
1. Create a constraint
1. Allocate fleet to infrastructure under some constraints
1. Plot the density profile of distances from vehicle destination to charging post

## Setting up the fleet, infrastructure, and constraint
First we create the fleet. It is simply a pandas data-frame with a specific set of
attributes (one attribute per column).  More attributes/columns can be added if
required for a specific application.  Here, we generate the fleet randomly with a
fairly minimal set. Run `evosim.fleet.random_fleet?` in a code-cell to see how to
parametrize the random generation. The ``seed`` argument is optional. We give it here
to ensure this notebook runs reproducibly during the tests and does not interfere with
version control.

In [ ]:
import numpy as np

import evosim

rng = np.random.default_rng(2)
fleet = evosim.fleet.random_fleet(1000, seed=rng)
fleet

Then we create the infrastructure of charging posts. Again, it is just a pandas
data-frame with a specific set of columns. And again, it is filled with mostly random
data. Arbitrarily, we've decided the capacity of the posts would range from 1 to 5 and
the occupancy from 0 to 2.

In [ ]:
infrastructure = evosim.charging_posts.random_charging_posts(
    len(fleet) // 3, capacity=5, occupancy=2, seed=rng
)
infrastructure

The random generation algorithm is smart enough to ensure the occupancy is always
smaller or equal to the capacity.

In [ ]:
assert (infrastructure.occupancy <= infrastructure.capacity).all()

Now we can create a ``matcher`` function: a constraint which allows or disallows
matching an electric vehicle with a charging post. Individual matchers can be found in
``evosim.matchers``. But there is also a factory function which helps to generate a
matcher function that is a combination of individual matchers. They are combined with
``and`` logic: all individual constraints must be true for the match to be okayed.

In [ ]:
matcher = evosim.matchers.factory(["socket_compatibility", "charger_compatibility"])

We can match a single vehicle to a single post

In [ ]:
matcher(fleet.loc[0], infrastructure.loc[1])

Or the full fleet to a single post

In [ ]:
matcher(fleet, infrastructure.loc[1])

Or a single vehicle to the full infrastructure

In [ ]:
matcher(fleet.loc[1], infrastructure)

Or we can match fleet and infrastructure element-wise: the first vehicle to the first
post, the second to the second, and so on. However, this element-wise comparison only
works if there are as many rows in fleet as in infrastructure. At time of writing, the
error thrown by pandas is not particularly enlightening:

In [ ]:
try:
    matcher(fleet, infrastructure)
except TypeError as e:
    print("Yup, we got an error: ", e)

But we can make the comparison work as follows:

In [ ]:
matcher(fleet.iloc[: len(infrastructure)], infrastructure)

## Running the allocation algorithm
The matcher can be used to constrain the allocation of vehicles to posts. Below, we
use the *random* allocator for this purpose. It allocates any vehicle to any post, as
long as the matcher okays it:

In [ ]:
fleet_allocation = evosim.allocators.random_allocator(fleet, infrastructure, matcher)
fleet_allocation

## Analysis and plots
We are now in a position to analyse the results. First we will take a look at the
unallocated vehicles. Then we will plot the distribution of distances from each
vehicles destination to each charging post.
Allocated and unallocated vehicles can be obtained as:

In [ ]:
unallocated_fleet = fleet[fleet_allocation.allocation.isna()]
allocated_fleet = fleet_allocation[fleet_allocation.allocation.notna()]

We can update the infrastructure to account for newly occupied posts. Note that the
infrastructure is copied with ``deep=True`` to ensure that changing the occupancy in
``new_infrastructure`` does not affect the original dataframe.

In [ ]:
new_infrastructure = infrastructure.copy(deep=True)
new_occupancy = allocated_fleet.groupby("allocation").allocation.count()
new_infrastructure.loc[new_occupancy.index, "occupancy"] += new_occupancy

Now we plot the number allocated and unallocated infrastructure and vehicles

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

output_notebook()
allocation_data = {
    "type": ["Initial Infrastructure", "Final Infrastructure", "Fleet"],
    "allocated": [
        infrastructure.occupancy.sum(),
        new_infrastructure.occupancy.sum(),
        len(allocated_fleet),
    ],
    "unallocated": [
        (infrastructure.capacity - infrastructure.occupancy).sum(),
        (new_infrastructure.capacity - new_infrastructure.occupancy).sum(),
        len(allocated_fleet),
    ],
}

barplot = figure(
    x_range=allocation_data["type"],
    height=200,
    title="Infrastructure and vehicle allocation",
)
barplot.vbar_stack(
    ["allocated", "unallocated"],
    x="type",
    width=0.9,
    source=allocation_data,
    color=["darkgrey", "lightgrey"],
    alpha=[1, 0.5],
    legend_label=["allocated", "unallocated"],
)
barplot.y_range.start = 0
barplot.legend.location = "top_left"
barplot.legend.orientation = "horizontal"
show(barplot)

To plot the density profile, we need the distance from the vehicle destinations to the
allocated charging posts. We use the ``evosim``'s objective function for that purpose.
The objective function expects two dataframe arguments, each with a "latitude" and a
"longitude" column. Hence, we first rename the destination columns of the
``allocated_fleet``. Also note that since the objective function compares dataframes,
they must conform to the same index.

In [ ]:
distance = evosim.objectives.distance(
    (
        allocated_fleet[["dest_lat", "dest_long"]].rename(
            columns=dict(dest_lat="latitude", dest_long="longitude")
        )
    ),
    infrastructure.loc[allocated_fleet.allocation].set_index(allocated_fleet.index),
)

density_plot = figure(title="Distance to charging post",)
y, x = np.histogram(distance, bins=20)
density_plot.line((x[1:] + x[:-1]) / 2, y)
show(density_plot)